In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
12,application_1575479622115_0013,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
11,application_1575479622115_0012,pyspark,idle,Link,Link,
12,application_1575479622115_0013,pyspark,idle,Link,Link,✔


In [3]:
sc.install_pypi_package("pandas")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/52/3f/f6a428599e0d4497e1595030965b5ba455fd8ade6e977e3c819973c4b41d/pandas-0.25.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

In [4]:
sc.install_pypi_package("matplotlib")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/4e/11/06958a2b895a3853206dea1fb2a5b11bf044f626f90745987612af9c8f2c/matplotlib-3.1.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f8/a1/5742b56282449b1c0968197f63eae486eca2c35dcd334bab75ad524e0de1/kiwisolver-1.1.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/c0/0c/fc2e007d9a992d997f04a80125b0f183da7fb554f1de701bbb70a8e7d479/pyparsing-2.4.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from pyspark.sql.functions import col, input_file_name, udf
from pyspark.sql.functions import sum as spark_sum

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

def run_sql_pandas(statement):
    return run_sql(statement).toPandas()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
raw_df = spark.read.json("s3://topic-sentiment-1/combined/*.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
raw_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- date_download: string (nullable = true)
 |-- date_modify: string (nullable = true)
 |-- date_publish: string (nullable = true)
 |-- description: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- language: string (nullable = true)
 |-- localpath: string (nullable = true)
 |-- source_domain: string (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- title_page: string (nullable = true)
 |-- title_rss: string (nullable = true)
 |-- url: string (nullable = true)

In [9]:
raw_df = raw_df.withColumn("source_file", input_file_name())

def file_only(path):
    return path.split('/')[-1]

file_only_udf = udf(file_only)
raw_df = raw_df.withColumn("source_file", file_only_udf(col("source_file")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
run_sql('drop database if exists topic cascade')
run_sql('create database topic')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [11]:
permanent_table_name = 'topic.articles'
raw_df.write.format("parquet").saveAsTable(permanent_table_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
by_file = run_sql_pandas('''
    SELECT source_file, source_domain, count(*) 
    FROM topic.articles
    GROUP by source_domain, source_file
''')
by_file.sort_values(by=['source_domain', 'source_file']).head(300)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                     source_file           source_domain  count(1)
49  2019-11-26_googlebot_lt.json  apnews.com              73877   
71  2019-12-03_default_lt.json    apnews.com              27032   
77  2019-11-26_googlebot_lt.json  axios.com               46207   
0   2019-12-03_default_lt.json    axios.com               32906   
5   2019-11-26_googlebot_lt.json  bbc.com                 64754   
26  2019-12-03_default_lt.json    bbc.com                 25254   
33  2019-11-26_googlebot_lt.json  bloomberg.com           4       
18  2019-11-26_googlebot_lt.json  boston.com              38238   
19  2019-12-03_default_lt.json    boston.com              11539   
34  2019-11-26_googlebot_lt.json  bostonglobe.com         91      
1   2019-12-03_default_lt.json    bostonglobe.com         88      
67  2019-11-26_googlebot_lt.json  breitbart.com           17988   
25  2019-12-03_default_lt.json    breitbart.com           17986   
57  2019-11-26_googlebot_lt.json  cbsnews.com             2125

In [19]:
run_sql_pandas('''
    SELECT source_domain, count(*) FROM (
    SELECT url, source_domain, count(*) FROM topic.articles
    GROUP BY url, source_domain
    HAVING count(*) > 1) a GROUP BY source_domain
''').sort_values(by='source_domain')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

             source_domain  count(1)
32  apnews.com              23646   
24  axios.com               32489   
16  bbc.com                 25219   
13  boston.com              3686    
29  breitbart.com           17475   
19  cbsnews.com             7399    
3   chicago.suntimes.com    43      
23  chicagotribune.com      15506   
30  chron.com               11056   
33  cnbc.com                1778    
21  denverpost.com          12266   
15  economist.com           55      
31  fivethirtyeight.com     13668   
17  foxnews.com             23681   
0   ft.com                  5760    
14  latimes.com             2058    
6   msnbc.com               869     
2   nationalreview.com      61      
9   nbcnews.com             9151    
20  npr.org                 252     
1   nytimes.com             4       
22  reason.com              1388    
5   reuters.com             1       
25  rt.com                  20283   
12  slate.com               8149    
8   theatlantic.com         7780    
4